# Creating a GPT Model

## Loading dataset and parameters

In [9]:
import pickle
import torch
import numpy as np
import torch.nn as nn
from torch.nn import functional as F
from contextlib import nullcontext

with open("./dataset/meta.pkl", 'rb') as f:
    meta = pickle.load(f)

vocab_size = meta["vocab_size"]
itos= meta["itos"]
stoi= meta["stoi"]


# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l]) 


dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device == 'cpu' else torch.amp.autocast(device_type=device, dtype=ptdtype)

# redefine batch generation
torch.manual_seed(1337)


def get_batch(split):
    # https://stackoverflow.com/questions/45132940/numpy-memmap-memory-usage-want-to-iterate-once/61472122#61472122
    if split == 'train':
        data = np.memmap("./dataset/train.bin", dtype=np.uint16, mode='r')
    else:
        data = np.memmap("./dataset/val.bin", dtype=np.uint16, mode='r')
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    if device == 'cuda' or device == 'mps':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y


# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss(m):
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx:
                logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out


xb, yb = get_batch('train')

In [11]:
class GPT(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx)  # (Batch, Time, Channel)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, _ = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C) 
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


m = GPT(vocab_size)
m.to(device)
logits, loss = m(xb, yb)


# generate data from the model
context = torch.zeros((1,1), dtype=torch.long, device= device) #1x1 tensor with 0 aka "\n"
result = m.generate(context, max_new_tokens=100)[0].tolist()
print(decode(result))


HXkx,rM
VTRPCynl3t cLYxRRXRU;elxFM3t-n$!pjRN-.kUdyMf.-:v$g?OsZ-Bi$iGnDLGfLUNNnPUrN-h
D-:RAU&rXV,snto


## Training of GPTModel


In [12]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [15]:
for iteration in range(max_iters): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    if iteration % eval_interval == 0 or iteration == max_iters - 1:
        losses = estimate_loss(m)
        print(f"step {iteration}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")


    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



step 0: train loss 4.7185, val loss 4.7147
step 100: train loss 4.5919, val loss 4.5899
step 200: train loss 4.4711, val loss 4.4722
step 300: train loss 4.3555, val loss 4.3520
step 400: train loss 4.2382, val loss 4.2438
step 500: train loss 4.1315, val loss 4.1318
step 600: train loss 4.0315, val loss 4.0369
step 700: train loss 3.9375, val loss 3.9354
step 800: train loss 3.8442, val loss 3.8473
step 900: train loss 3.7569, val loss 3.7586
step 1000: train loss 3.6722, val loss 3.6747
step 1100: train loss 3.5964, val loss 3.5969
step 1200: train loss 3.5211, val loss 3.5222
step 1300: train loss 3.4489, val loss 3.4510
step 1400: train loss 3.3860, val loss 3.3911
step 1500: train loss 3.3204, val loss 3.3306
step 1600: train loss 3.2640, val loss 3.2712
step 1700: train loss 3.2060, val loss 3.2152
step 1800: train loss 3.1597, val loss 3.1629
step 1900: train loss 3.1128, val loss 3.1149
step 2000: train loss 3.0656, val loss 3.0644
step 2100: train loss 3.0170, val loss 3.0289


In [17]:
# generate data from the model
context = torch.zeros((1,1), dtype=torch.long, device= device) #1x1 tensor with 0 aka "\n"
result = m.generate(context, max_new_tokens=500)[0].tolist()
print(decode(result))


s!


NVFwcourd ithe oriTr ico,
SNU&HAg:
She s, nard h wild&N: y he thelo y,
Tovet f I I re he ule VORCowiblistomye bu r t XJUTonqo t sers allet.

FlliwcFrestS:
IAs youn wans mar KI ws, t ILamm itizilsKCUC3ns t, Pers? mefea!
HUSO lita CELE:
RTh mo gomaveld caBY;
Flly hour ou tok d FO:ChaJrox
ASOSe-haJEOLorse lon le s forenBRe ng p r alarn yos tue s
W
UJoe o:
Lge InLenoryouMin, sthermeeyorcant o,
Ore at t ld, bTyour downetCoulle
TINoncosanFot bry oilleIOWhe thil; m e tDedefriccak!
P, AUCUThiane te


## Mathematical Trick for Self Attention

At the moment, the tokens does not communicate with each other. We need to change that. But we need to ensure that not any information of the "future" is getting passed to past - because we want do predict it. So the 5th token should not talk to tokens on the 6th, 7th and 8th location. Information should only flow from the previous timestamp to the current.

A simple way to do this is to **get the average of all preceding tokens** to have a single feature for calculation. We will lose a lot of information about spacial relation ship of tokens, but thats okay for lecturing purposes. 

### Version 1

In [20]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [18]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [19]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)